In [1]:
## BUSINESS USE CASE -- PLANE CRASHES HAPPEN!! HERE'S HOW YOU CAN AVOID DYING!!

In [2]:
import csv
import pickle
import pandas as pd
import numpy as np
import patsy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve, auc
from sklearn.model_selection import learning_curve, KFold, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from collections import Counter
import matplotlib.style as style
style.use('fivethirtyeight')
%matplotlib inline

from __future__ import print_function, division

# Get pandas and postgres to work together
import psycopg2 as pg
import pandas.io.sql as pd_sql
plt.rc('font', size=18) 

/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
with open('airline_crash.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [4]:
state_df = pd.concat([df['Event_Id'], df['Location'].str.split(', ', expand=True)], axis=1)

In [5]:
state_df['City'] = state_df[0]
state_df['State'] = state_df[1]

state_df = state_df.drop([0,1,2,3,4],1)

In [7]:
df.reset_index(inplace=True)
state_df.reset_index(inplace=True)

In [ ]:
df = pd.merge(df, state_df, on='index')

df = df.drop(['Location'],1)

In [ ]:
df.Latitude = df.Latitude.astype('float')
df.Longitude = df.Longitude.astype('float')
df.Total_Fatal_Injuries = df.Total_Fatal_Injuries.astype('float')
df.Total_Serious_Injuries = df.Total_Serious_Injuries.astype('float')
df.Total_Minor_Injuries = df.Total_Minor_Injuries.astype('float')
df.Total_Uninjured = df.Total_Uninjured.astype('float')
df.Number_Engines = df.Number_Engines.astype('float')

df['Event_Date'] = pd.to_datetime(df['Event_Date'])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.Broad_Phase_of_Flight.value_counts()

In [ ]:
df.Broad_Phase_of_Flight.replace('CRUISE', 'INFLIGHT', inplace=True)
df.Broad_Phase_of_Flight.replace('MANEUVERING', 'INFLIGHT', inplace=True)
df.Broad_Phase_of_Flight.replace('GO-AROUND', 'INFLIGHT', inplace=True)
df.Broad_Phase_of_Flight.replace('APPROACH', 'LANDING', inplace=True)
df.Broad_Phase_of_Flight.replace('DESCENT', 'INFLIGHT', inplace=True)
df.Broad_Phase_of_Flight.replace('CLIMB', 'INFLIGHT', inplace=True)
df.Broad_Phase_of_Flight.replace('STANDING', 'TAXI', inplace=True)

In [ ]:
df.Broad_Phase_of_Flight.value_counts()

In [ ]:
df = df[df['Broad_Phase_of_Flight'] != 'OTHER']
df = df[df['Broad_Phase_of_Flight'] != 'UNKNOWN']

In [ ]:
df.Broad_Phase_of_Flight.value_counts()

In [ ]:
df = df.rename(columns={'Event_Id_x': 'Event_Id'})

df = df.drop(['Event_Id_y'], axis=1)

In [ ]:
larger_df = df.dropna(subset=['Investigation_Type','Event_Date',
              'Country', 'Injury_Severity', 'Amateur_Built', 'Number_Engines', 'Engine_Type',
              'Flight_Purpose', 'Total_Fatal_Injuries', 'Total_Serious_Injuries',
              'Total_Minor_Injuries', 'Total_Uninjured', 'Weather_Condition', 'Broad_Phase_of_Flight','City','State'])

larger_df.info()

In [ ]:
larger_df.Aircraft_Category.value_counts()

In [ ]:
larger_df.Weather_Condition.value_counts()

In [ ]:
larger_df = larger_df[larger_df['Weather_Condition'] != 'UNK']
larger_df = larger_df[larger_df['Aircraft_Category'] == 'Airplane']

In [ ]:
larger_df.info()

In [ ]:
larger_df = larger_df[['Event_Id','Investigation_Type','Event_Date',
              'Country', 'Injury_Severity',
              'Aircraft_Category', 'Amateur_Built', 'Number_Engines', 'Engine_Type',
              'Flight_Purpose', 'Total_Fatal_Injuries', 'Total_Serious_Injuries',
              'Total_Minor_Injuries', 'Total_Uninjured', 'Weather_Condition', 'Broad_Phase_of_Flight','City','State']]

In [ ]:
larger_df['Total_Passengers'] = (larger_df['Total_Fatal_Injuries'] + larger_df['Total_Serious_Injuries'] + 
                                larger_df['Total_Minor_Injuries'] + larger_df['Total_Uninjured'])